# Process json file generated by labelme

Our 'json/' folder contains our annotations (made with labelme, which is an open source software) of Test400 images. we leverage json data to generate .png labels like images in 'glaucoma/Disc_Cup_Masks/'.

In [51]:
import os
import json
import numpy as np
from PIL import Image

folder = 'json/'

for root, dirs, files in os.walk(folder):
    for file in files:
#         print(file)
        with open(os.path.join(root, file), 'r') as f:
            data = json.load(f)
#         print(data)
        height, width = data['imageHeight'], data['imageWidth']
#         print(height, width)
        for label in data['shapes']:
#             print(label)
            if label['label'] == 'cup': cup = label['points']
            else: disc = label['points']
        
        assert cup != None and disc != None
        
        if disc[-1] != disc[0]: disc.append(disc[0])  # close polygon
        if cup[-1] != cup[0]: cup.append(cup[0])
#         print(disc)
#         print(cup)
        
        name = data['imagePath'].split('\\')[-1].split('.')[0]
        print('processing {}, height = {}, width = {}'.format(name, height, width))
#         imagePath = os.path.join('Test400', data['imagePath'].split('\\')[-1])
#         image = np.array(Image.open(imagePath))
        cup_ = np.zeros((width, height), int)
        disc_ = np.zeros((width, height), int)
        label_ = np.zeros((width, height), int)
        
#         print('\nprocessing disc area')
        for i in range(1, len(disc)):
            x1, y1, x2, y2 = disc[i-1][0], disc[i-1][1], disc[i][0], disc[i][1]
#             print(x1, y1, x2, y2)
            if (x1 != x2):
                k = (y2-y1) / (x2-x1)
                b = y1 - k*x1
                for x in range(min(x1, x2), max(x1, x2)):
                    for y in range(round(k*x + b)):
                        disc_[y, x] ^= 1
        
#         print('\nprocessing cup area')
        for i in range(1, len(cup)):
            x1, y1, x2, y2 = cup[i-1][0], cup[i-1][1], cup[i][0], cup[i][1]
#             print(x1, y1, x2, y2)
            if (x1 != x2):
                k = (y2-y1) / (x2-x1)
                b = y1 - k*x1
                for x in range(min(x1, x2), max(x1, x2)):
                    for y in range(round(k*x + b)):
                        cup_[y, x] ^= 1
        
        label_[...] = 255
        label_[disc_ == 1] = 128
        label_[cup_ == 1] = 0
#         print(label_[label_ != 0])
        
        Image.fromarray(np.uint8(label_)).save(os.path.join('glaucoma/gt400', name + '.bmp'))
        

processing T0260, height = 1634, width = 1634
processing T0325, height = 1634, width = 1634
processing T0237, height = 1634, width = 1634
processing T0372, height = 1634, width = 1634
processing T0221, height = 1634, width = 1634
processing T0364, height = 1634, width = 1634
processing T0276, height = 1634, width = 1634
processing T0299, height = 1634, width = 1634
processing T0333, height = 1634, width = 1634
processing T0348, height = 1634, width = 1634
processing T0309, height = 1634, width = 1634
processing T0283, height = 1634, width = 1634
processing T0329, height = 1634, width = 1634
processing T0391, height = 1634, width = 1634
processing T0387, height = 1634, width = 1634
processing T0368, height = 1634, width = 1634
processing T0295, height = 1634, width = 1634
processing T0344, height = 1634, width = 1634
processing T0201, height = 1634, width = 1634
processing T0313, height = 1634, width = 1634
processing T0256, height = 1634, width = 1634
processing T0305, height = 1634, w

processing T0279, height = 1634, width = 1634
processing T0202, height = 1634, width = 1634
processing T0347, height = 1634, width = 1634
processing T0255, height = 1634, width = 1634
processing T0310, height = 1634, width = 1634
processing T0243, height = 1634, width = 1634
processing T0306, height = 1634, width = 1634
processing T0214, height = 1634, width = 1634
processing T0351, height = 1634, width = 1634
processing T0326, height = 1634, width = 1634
processing T0263, height = 1634, width = 1634
processing T0371, height = 1634, width = 1634
processing T0234, height = 1634, width = 1634
processing T0367, height = 1634, width = 1634
processing T0222, height = 1634, width = 1634
processing T0388, height = 1634, width = 1634
processing T0330, height = 1634, width = 1634
processing T0275, height = 1634, width = 1634
processing T0259, height = 1634, width = 1634
processing T0218, height = 1634, width = 1634


```
y = kx + b

```

Note:
1. PIL defaults to the origin being in the top-left
2. x and y is reversed

# Generate txt files and preprocess images and labels

In [52]:
import os
import random
import numpy as np
from PIL import Image

cup_area = 0
disc_area = 0
other_area = 0


def image2png(img_path, is_label=False, height=240, width=240):
    """
    :param img_path: a string, path to original image
    :param height: int, wanted image height
    :param width: int, wanted image width
    :param is_label: bool, process label
    :return: a string, path to converted image
    """
    global cup_area, disc_area, other_area
    img = Image.open(img_path).resize((width, height))

    if is_label:
        img = np.array(img)
        rows, cols = img.shape
        for r in range(rows):
            for c in range(cols):
                if img[r][c] < 100: img[r][c] = 0; cup_area += 1
                elif img[r][c] < 200: img[r][c] = 1; disc_area += 1
                else: img[r][c] = 2; other_area += 1
        img = Image.fromarray(img)

    png = os.path.splitext(img_path)[0] + '.png'
    img.save(png)

    return png


def preprocess(ALREADY_HAVE_PNG):
    
    data_folder  = 'glaucoma/Training400/Training400/'
    label_folder = 'glaucoma/Disc_Cup_Masks/'
    test_folder  = 'glaucoma/Test400/'
    gt_folder    = 'glaucoma/gt400/'  # our annotations

    def _preprocess(folder, is_label):
        paths = []
        for root, dirs, files in os.walk(folder):
            files.sort()
            for name in files:
                path = os.path.join(root, name)
                ext = os.path.splitext(name)[-1]
                if ALREADY_HAVE_PNG and ext == '.png':
                    paths.append(path)
                elif not ALREADY_HAVE_PNG and (ext == '.jpg' or ext == '.bmp'):
                    print('processing', path)
                    paths.append(image2png(path, is_label))
        return paths

    print('processing training set ...')
    data_dir = _preprocess(data_folder, is_label=False)

    print('processing label set ...')
    label_dir = _preprocess(label_folder, is_label=True)
    
    print('processing test set ...')
    test_dir = _preprocess(test_folder, is_label=False)
    
    print('processing gt set ...')
    gt_dir = _preprocess(gt_folder, is_label=True)

    training = list(zip(data_dir, label_dir))
    test = list(zip(test_dir, gt_dir))
    random.shuffle(training)  # shuffle training set
    random.shuffle(test)  # shuffle testing set
    print('number of training images: {}'.format(len(training)))
    print('number of testing images: {}'.format(len(test)))

    train, val = [], []
    for i, path in enumerate(training):
        # 1/5 for validation, 4/5 for training
        if i < len(training) / 5: val.append(path)
        else: train.append(path)

    with open('train.txt', 'w') as f:
        for image, label in train:
            f.write(image + ' ' + label + '\n')

    with open('val.txt', 'w') as f:
        for image, label in val:
            f.write(image + ' ' + label + '\n')
    
    with open('test.txt', 'w') as f:
        for image, label in test:  # test_dir is no use for now
            f.write(image + ' ' + label + '\n')


preprocess(ALREADY_HAVE_PNG=False)

print('cup_area: ', cup_area)
print('disc_area: ', disc_area)
print('other_area: ', other_area)

processing training set ...
processing glaucoma/Training400/Training400/Glaucoma/g0001.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0002.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0003.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0004.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0005.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0006.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0007.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0008.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0009.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0010.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0011.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0012.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0013.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0014.jpg
processing glaucoma/Training400/Training400/Glaucoma/g0015.jpg
processing glaucoma/Trainin

processing glaucoma/Training400/Training400/Non-Glaucoma/n0086.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0087.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0088.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0089.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0090.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0091.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0092.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0093.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0094.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0095.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0096.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0097.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0098.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0099.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0100

processing glaucoma/Training400/Training400/Non-Glaucoma/n0209.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0210.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0211.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0212.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0213.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0214.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0215.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0216.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0217.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0218.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0219.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0220.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0221.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0222.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0223

processing glaucoma/Training400/Training400/Non-Glaucoma/n0333.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0334.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0335.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0336.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0337.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0338.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0339.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0340.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0341.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0342.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0343.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0344.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0345.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0346.jpg
processing glaucoma/Training400/Training400/Non-Glaucoma/n0347

processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0073.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0074.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0075.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0076.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0077.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0078.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0079.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0080.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0081.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0082.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0083.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0084.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0085.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0086.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0087.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0088.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0089.bmp
processing gla

processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0215.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0216.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0217.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0218.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0219.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0220.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0221.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0222.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0223.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0224.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0225.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0226.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0227.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0228.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0229.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0230.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0231.bmp
processing gla

processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0357.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0358.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0359.bmp
processing glaucoma/Disc_Cup_Masks/Non-Glaucoma/n0360.bmp
processing test set ...
processing glaucoma/Test400/T0201.jpg
processing glaucoma/Test400/T0202.jpg
processing glaucoma/Test400/T0203.jpg
processing glaucoma/Test400/T0204.jpg
processing glaucoma/Test400/T0205.jpg
processing glaucoma/Test400/T0206.jpg
processing glaucoma/Test400/T0207.jpg
processing glaucoma/Test400/T0208.jpg
processing glaucoma/Test400/T0209.jpg
processing glaucoma/Test400/T0210.jpg
processing glaucoma/Test400/T0211.jpg
processing glaucoma/Test400/T0212.jpg
processing glaucoma/Test400/T0213.jpg
processing glaucoma/Test400/T0214.jpg
processing glaucoma/Test400/T0215.jpg
processing glaucoma/Test400/T0216.jpg
processing glaucoma/Test400/T0217.jpg
processing glaucoma/Test400/T0218.jpg
processing glaucoma/Test400/T0219.jpg
processing glaucoma/Te

processing glaucoma/gt400/T0210.bmp
processing glaucoma/gt400/T0211.bmp
processing glaucoma/gt400/T0212.bmp
processing glaucoma/gt400/T0213.bmp
processing glaucoma/gt400/T0214.bmp
processing glaucoma/gt400/T0215.bmp
processing glaucoma/gt400/T0216.bmp
processing glaucoma/gt400/T0217.bmp
processing glaucoma/gt400/T0218.bmp
processing glaucoma/gt400/T0219.bmp
processing glaucoma/gt400/T0220.bmp
processing glaucoma/gt400/T0221.bmp
processing glaucoma/gt400/T0222.bmp
processing glaucoma/gt400/T0223.bmp
processing glaucoma/gt400/T0224.bmp
processing glaucoma/gt400/T0225.bmp
processing glaucoma/gt400/T0226.bmp
processing glaucoma/gt400/T0227.bmp
processing glaucoma/gt400/T0228.bmp
processing glaucoma/gt400/T0229.bmp
processing glaucoma/gt400/T0230.bmp
processing glaucoma/gt400/T0231.bmp
processing glaucoma/gt400/T0232.bmp
processing glaucoma/gt400/T0233.bmp
processing glaucoma/gt400/T0234.bmp
processing glaucoma/gt400/T0235.bmp
processing glaucoma/gt400/T0236.bmp
processing glaucoma/gt400/T0